### Import de Bibliotecas

In [68]:
#Automação WEB
from selenium import webdriver
import chromedriver_autoinstaller
from selenium.webdriver.common.by import By
from time import sleep
from selenium.webdriver.common.keys import Keys
from pyautogui import alert, confirm

#Alerta
import tkinter as tk
from tkinter import messagebox
import sys

#Organizar/Extrair_ZIP/Load Arquivos
import os
from pathlib import Path
import shutil
from zipfile import ZipFile
import pandas as pd

#Conexão ao DB
from sqlalchemy import create_engine, text
import urllib.parse

#Envio de E-mail
import smtplib
import email.message

### Alerta de Script

In [69]:
def confirm_dialog():
    # Criando a janela principal (oculta)
    root = tk.Tk()
    root.withdraw()  # Ocultar a janela principal

    # Exibir a caixa de diálogo de confirmação
    result = messagebox.askquestion("Script", "Script de extração/inserção das Bases.", icon='question')

    # Verificar a resposta
    if result == 'yes':
        print("Confirmado")
    else:
        print("Cancelado")
        sys.exit()  # Interromper o script se "Cancelar" for clicado

# Chamar a função
confirm_dialog()

# Resto do script continua se confirmado
print("O script continua executando...")

Confirmado
O script continua executando...


### Web Scraping

In [70]:
chromedriver_autoinstaller.install()
Navegador = webdriver.Chrome()

Link = "https://www.kaggle.com/datasets/thedevastator/the-ultimate-netflix-tv-shows-and-movies-dataset"
Navegador.get(Link)
sleep(1)

Button_Sign_In = Navegador.find_element('xpath', '//*[@id="site-container"]/div/div[4]/div[2]/div[2]/div/div[1]/a/button')
Button_Sign_In.click()
sleep(1)

Button_Email = Navegador.find_element('xpath', '//*[@id="site-content"]/div[2]/div/div/div[1]/form/div/div/div[1]/button[2]')
Button_Email.click()
sleep(1)

Email_Address = Navegador.find_element('xpath', '//*[@id=":r7:"]')
Email_Address.send_keys('emiliano.junqueira@gmail.com')

Password = Navegador.find_element('xpath', '//*[@id=":r8:"]')
Password .send_keys('K@ggle0051')

Sign_In = Navegador.find_element('xpath', '//*[@id="site-content"]/div[2]/div[1]/div/div[1]/form/div/div[4]/button[2]')
Sign_In.click()
sleep(5)

Download = Navegador.find_element('xpath', '//*[@id="site-content"]/div[2]/div/div[2]/div/div[2]/div[1]/div/div[2]/div[1]/button[1]')
Download.click()
sleep(5)

Navegador.quit()

### Organizar e Extrair o arquivo ZIP

In [71]:
Origin = ('C:\\Users\\Miza\\Downloads\\archive.zip')
Destiny = Path('C:\\Users\\Miza\\Desktop\\CoderHouse\\ProjetoSQL')

shutil.copy2(Origin, Destiny)

if os.path.exists(Origin):
    os.remove(Origin)

with ZipFile("archive.zip", "r") as zip:
    zip.extractall()

### Conexão ao MySQL

In [72]:
# Parâmetros de conexão
usuario = "root"
senha = "Are@0051"
host = "localhost"  # Ou o IP do servidor
nome_do_banco = "netflix"

# URL-encode da senha para tratar caracteres especiais
senha_codificada = urllib.parse.quote_plus(senha)

# String de conexão com MySQL
conexao_string = f"mysql+pymysql://{usuario}:{senha_codificada}@{host}/{nome_do_banco}"

# Criação do engine de conexão
engine = create_engine(conexao_string)

### Truncar e Sobrescrever tabelas no DB

In [73]:
# Função para sobrescrever a tabela no banco de dados
def sobrescrever_tabela(tabela_nome, df):
    with engine.begin() as connection:
        # Limpar a tabela (TRUNCATE mantém a estrutura e chaves)
        connection.execute(text(f"TRUNCATE TABLE {tabela_nome}"))
        
        # Inserir os novos dados
        df.to_sql(tabela_nome, con=connection, if_exists='append', index=False)

# Lista de tabelas e arquivos CSV correspondentes
tabelas_arquivos = {

    "best_movie_by_year_netflix"  : "C:/Users/Miza/Desktop/CoderHouse/ProjetoSQL/Best Movie by Year Netflix.csv",
    "best_movies_netflix"         : "C:/Users/Miza/Desktop/CoderHouse/ProjetoSQL/Best Movies Netflix.csv",
    "best_show_by_year_netflix"   : "C:/Users/Miza/Desktop/CoderHouse/ProjetoSQL/Best Show by Year Netflix.csv",
    "best_shows_netflix"          : "C:/Users/Miza/Desktop/CoderHouse/ProjetoSQL/Best Shows Netflix.csv",
    "raw_credits"                 : "C:/Users/Miza/Desktop/CoderHouse/ProjetoSQL/raw_credits.csv",
    "raw_titles"                  : "C:/Users/Miza/Desktop/CoderHouse/ProjetoSQL/raw_titles.csv"
    
}

# Itera sobre as tabelas e seus arquivos correspondentes
for tabela, arquivo in tabelas_arquivos.items():
    # Carrega o CSV em um DataFrame
    df = pd.read_csv(arquivo)
    
    # Sobrescreve a tabela no banco de dados
    sobrescrever_tabela(tabela, df)

print("Tabelas sobrescritas com sucesso!")


engine.dispose()

Tabelas sobrescritas com sucesso!


### Alerta de Fim do Script 

In [74]:
# Alerta de conclusão
def alert_end():
    root = tk.Tk()
    root.withdraw()  # Ocultar a janela principal
    messagebox.showinfo("Conclusão", "O script foi concluído com sucesso!")

# Chamar o alerta ao final do script
alert_end()

# O script finaliza
print("O script foi concluído.")

O script foi concluído.
